In [ ]:
!pip install transformers[torch]
!pip install sentencepiece

In [ ]:
import os

import pandas as pd
import torch
import transformers

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

**テキスト生成**

In [ ]:
# rinna社の事前学習済みGPTモデルのチェックポイント名
model_checkpoint = "rinna/japanese-gpt2-medium"
# rinna社の事前学習済みGPTモデルのトークナイザーを取得
tokenizer = transformers.AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
print(len(tokenizer.get_vocab()))  # トークナイザーの辞書の単語数
print(tokenizer.pad_token_id)  # トークナイザーの辞書に登録されているpaddingのID
print(tokenizer.bos_token_id)  # トークナイザーの辞書に登録されているBoS(Begin of Sentence)のID
print(tokenizer.eos_token_id)  # トークナイザーの辞書に登録されているEoS(End of Sentence)のID
print(tokenizer.unk_token_id)  # トークナイザーの辞書に登録されている不明語のID

In [ ]:
text_input_data = "お勧めのライフハックと言えば、"
tokenized_text_input_data = tokenizer(text_input_data,
                                      padding=True,
                                      truncation=True,
                                      add_special_tokens=False,
                                      return_tensors="pt").to(device)

In [ ]:
print(tokenized_text_input_data.data)  # トークン化した文章データの中身

In [ ]:
# rinna社の事前学習済みGPTモデルのニューラルネットワークモデルを取得
nn_model = transformers.AutoModelForCausalLM.from_pretrained(model_checkpoint).to(device)

In [ ]:
# テキスト生成
with torch.no_grad():
    nn_model.eval()
    output = nn_model.generate(input_ids=tokenized_text_input_data.data["input_ids"],  # インプットとなるトークンを2次元行列で格納
                               attention_mask=tokenized_text_input_data.data["attention_mask"],  # アテンションマスクを2次元行列で格納
                               max_length=100,  # 生成されるテキストの最大長
                               min_length=20,  # 生成されるテキストの最小長
                               num_return_sequences=3,  # 生成される文章の数
                               do_sample=True,  # サンプリングをするかどうか
                               top_k=500,  # ランダムなk個の単語の中から生成される単語を選択
                               top_p=0.95,  # 累積確率を制限する割合(0<x<1の範囲)
                               pad_token_id=tokenizer.pad_token_id,  # パディングのトークンを指定
                               bos_token_id=tokenizer.bos_token_id,  # BoS(Begin of Sentence)のトークンを指定
                               eos_token_id=tokenizer.eos_token_id,  # EoS(End of Sentence)のトークンを指定
                               bad_words_ids=[[tokenizer.unk_token_id]])  # 生成しない単語のトークンをリスト of リストの形で指定
print(output)
print(output.shape)

In [ ]:
generated_text_dict = {}
for i in range(output.shape[0]):
    output_generated_text = tokenizer.convert_ids_to_tokens(output[i].detach().cpu().numpy())  # テンソルから微分計算を外して、cpu処理にして、numpyへ変換(※1次元ベクトルである事)した後に、id2value変換
    print(output_generated_text)
    print(type(output_generated_text))
    generated_text = "".join(output_generated_text)
    generated_text = generated_text.replace("▁", "")
    generated_text = generated_text.replace("</s>", "")
    generated_text = generated_text.replace("[PAD]", "")
    generated_text_dict["{a}つ目".format(a=i+1)] = generated_text

In [ ]:
generated_text_dict

**学習(Fine Tuning)**

In [ ]:
# FineTuningのための学習データを収集
!wget https://www.rondhuit.com/download/ldcc-20140209.tar.gz
!tar zxvf ldcc-20140209.tar.gz

In [ ]:
# フォルダ内のテキストファイルからタイトルを取得して、タイトルリストを作成する関数
def get_title_list(path):
    title_list = []
    filenames = os.listdir(path)
    for filename in filenames:
        with open(path+filename) as f:  # テキストファイルの読み込み
            title = f.readlines()[2].strip()
            title_list.append(title)
    return title_list

In [ ]:
# 「it-life-hack」フォルダ内にあるテキストファイルからタイトルリストのDataFrameを作成
df = pd.DataFrame(columns=["category",
                           "sentence"])
title_list = get_title_list("text/it-life-hack/")
for title in title_list:
    df_temp = pd.DataFrame(data={"category": ["it-life-hack"],
                                 "sentence": [title]})
    df = pd.concat(objs=[df, df_temp],
                   ignore_index=True)
# DataFrameの中身をシャッフル
df = df.sample(frac=1)
# DataFrameの中身の最初から80%を学習用、末尾から20%を検証用として、csvファイルを作成
num = len(df)
df.iloc[:int(num*0.8), :].to_csv("./train.csv", sep=",", index=False)
df.iloc[-int(num*0.2):, :].to_csv("./valid.csv", sep=",", index=False)

In [ ]:
# 学習用csvファイルをDataFrameとして取得
train_df = pd.read_csv("./train.csv", encoding="utf-8")
# 「sentence」列だけを取得して、1行1データの形のテキストファイルを新しく作成する
for i in range(len(train_df)):
    sentence = train_df.iloc[i, 1]  # forループで1行ずつ取得で、列は要素1でsentence列(要素0だとcategory列)
    with open("./train_sentence.txt", "a") as f:
        f.write(sentence + "\n")

In [ ]:
# 学習データのDataset作成
fine_tuning_train_dataset = transformers.TextDataset(tokenizer=tokenizer,
                                                     file_path="./train_sentence.txt",  # 学習データのファイルの場所
                                                     block_size=128)

In [ ]:
# Fine Tuningの際の学習データのマスクに関する設定
fine_tuning_collator = transformers.DataCollatorForLanguageModeling(tokenizer=tokenizer,
                                                                    mlm=False)

In [ ]:
# Fine Tuningに関する設定
fine_tuning_args = transformers.TrainingArguments(output_dir="./generate",  # ファイルの出力先
                                                  overwrite_output_dir=True,  # 出力先フォルダを上書きするかどうか
                                                  evaluation_strategy="epoch",  # 評価関数を計算するタイミング
                                                  logging_steps=100,  # ログを出すタイミング
                                                  log_level="error",  # ログレベル
                                                  save_strategy="no",  # チェックポイントを保存するタイミング
                                                  save_total_limit=None,  # チェックポイントを保存する数
                                                  learning_rate=2e-5,  # 学習率
                                                  weight_decay=0.01,  # 減衰率
                                                  per_device_train_batch_size=8,  # 学習の時のミニバッチ数
                                                  fp16=True,  # 16ビット計算
                                                  num_train_epochs=3)  # 学習エポック数

In [ ]:
# Fine Tuning
nn_model_trainer = transformers.Trainer(model=nn_model,
                                        args=fine_tuning_args,
                                        data_collator=fine_tuning_collator,
                                        train_dataset=fine_tuning_train_dataset)
nn_model.train()
nn_model_trainer.train()

Step,Training Loss


TrainOutput(global_step=51, training_loss=3.607676487342984, metrics={'train_runtime': 2245.8705, 'train_samples_per_second': 0.174, 'train_steps_per_second': 0.023, 'total_flos': 90548317716480.0, 'train_loss': 3.607676487342984, 'epoch': 3.0})

In [ ]:
# Fine Tuningの結果をセーブ
nn_model_trainer.save_model("./GPT-model-after-FineTuning")
# Fine Tuning時の損失値や評価値などの情報をセーブ
nn_model_trainer.save_state()

**Fine Tuning後のテキスト生成**

In [ ]:
# rinna社のトークナイザーと、FineTuning後の事前学習済みGPTモデルのニューラルネットワークモデルをロードして取得
model_checkpoint = "rinna/japanese-gpt2-medium"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_checkpoint)
nn_model_after_ft = transformers.AutoModelForCausalLM.from_pretrained("./GPT-model-after-FineTuning").to(device)

In [ ]:
# テキスト生成
text_input_data = "お勧めのライフハックと言えば、"
tokenized_text_input_data = tokenizer(text_input_data,
                                      padding=True,
                                      truncation=True,
                                      add_special_tokens=False,
                                      return_tensors="pt").to(device)
with torch.no_grad():
    nn_model_after_ft.eval()
    output_after_ft = nn_model_after_ft.generate(input_ids=tokenized_text_input_data.data["input_ids"],  # インプットとなるトークンを2次元行列で格納
                                                 attention_mask=tokenized_text_input_data.data["attention_mask"],  # アテンションマスクを2次元行列で格納
                                                 max_length=100,  # 生成されるテキストの最大長
                                                 min_length=20,  # 生成されるテキストの最小長
                                                 num_return_sequences=3,  # 生成される文章の数
                                                 do_sample=True,  # サンプリングをするかどうか
                                                 top_k=500,  # ランダムなk個の単語の中から生成される単語を選択
                                                 top_p=0.95,  # 累積確率を制限する割合(0<x<1の範囲)
                                                 pad_token_id=tokenizer.pad_token_id,  # パディングのトークンを指定
                                                 bos_token_id=tokenizer.bos_token_id,  # BoS(Begin of Sentence)のトークンを指定
                                                 eos_token_id=tokenizer.eos_token_id,  # EoS(End of Sentence)のトークンを指定
                                                 bad_words_ids=[[tokenizer.unk_token_id]])  # 生成しない単語のトークンをリスト of リストの形で指定
print(output_after_ft)
print(output_after_ft.shape)

In [ ]:
generated_text_dict = {}
for i in range(output_after_ft.shape[0]):
    output_generated_text_after_ft = tokenizer.convert_ids_to_tokens(output_after_ft[i].detach().cpu().numpy())  # テンソルから微分計算を外して、cpu処理にして、numpyへ変換(※1次元ベクトルである事)した後に、id2value変換
    print(output_generated_text_after_ft)
    print(type(output_generated_text_after_ft))
    generated_text = "".join(output_generated_text_after_ft)
    generated_text = generated_text.replace("▁", "")
    generated_text = generated_text.replace("</s>", "")
    generated_text = generated_text.replace("[PAD]", "")
    generated_text_dict["{a}つ目".format(a=i+1)] = generated_text

In [ ]:
generated_text_dict

{'1つ目': 'お勧めのライフハックと言えば、やはり「シェア」「モノの場所」「モバイルのスゴイところ」なのだが、これも全く使っていなかった。しかし本当に使ってみると便利でやめられなくなる。と言うか使い道がない。はてなもはてなダイアリーも便利だけど、それに比べてたしかにblogなんかにいいよね。あと気になった「今すぐ笑わせろ」企画も。はてなには本当にこんなことが得意',
 '2つ目': 'お勧めのライフハックと言えば、ios向けアプリ「夢の案内人!旅の指さし会話帳～イタリア編」が人気を集めていることを紹介しました。また、海外の“移動端末”はどうなったか?外出先で通信費用を抑える「acアダプター」とは?昨年12月の「ios7.1.1」のリリースから2年が経った現在でも、多くのユーザーから注目されているのが「“マップ移動”アプリ」',
 '3つ目': 'お勧めのライフハックと言えば、この記事を是非チェックしてみてください。古い記事を読んでほしくない人は読み飛ばしましょう!こちらの記事では、いま話題の「フリック操作」を実践するテクニックを紹介します。android端末の“見た目の美しさ”に気付いていますか?「外観をキレイに仕上げる最強アプリ」スマホのフォルダやフォルダを整理できる【知っ得!虎の巻'}

In [ ]:
# モデルを表示
print(nn_model_after_ft)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(32000, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=32000, bias=False)
)


In [ ]:
# モデルの重み、バイアスのパラメータ数
params = 0
for p in nn_model_after_ft.parameters():
    if p.requires_grad:
        params += p.numel()
print(params)

336128000
